# Create model arquitecture

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.data import Dataset
from tensorflow import feature_column
from preprocess_tf import preprocessing_fn

In [2]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target_name, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    targets = dataframe.pop(target_name)
    ds = Dataset.from_tensor_slices((dict(dataframe), targets))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
        ds = ds.batch(batch_size)
    return ds

In [3]:
def set_numerical_feature(name):
    
    numerical_feature = feature_column.numeric_column(name)
    
    return numerical_feature
    
def set_one_hot_feature(name, data):
    one_hot_feature = feature_column.categorical_column_with_vocabulary_list(name, data[name].unique().tolist())
    one_hot_feature = feature_column.indicator_column(one_hot_feature)
    
    return one_hot_feature

def set_embedding_feature(name, data, dims):
    embedding_feature = feature_column.categorical_column_with_vocabulary_list(name, data[name].unique().tolist())
    embedding_feature = feature_column.embedding_column(embedding_feature, dimension=dims)
    
    return embedding_feature

In [4]:
def feature_columns(data, dataset):
    
    feature_columns = []

    # numeric cols
    for header in ['yearOfRegistration', 'powerPS', 'kilometer']:
        feature_columns.append(set_numerical_feature(header))

    feature_columns.append(set_one_hot_feature('abtest', data))
    
    feature_columns.append(set_embedding_feature('vehicleType', data, 4))
    
    feature_columns.append(set_one_hot_feature('gearbox', data))
    
    feature_columns.append(set_embedding_feature('model', data, 8))
    
    feature_columns.append(set_one_hot_feature('fuelType', data))
    
    feature_columns.append(set_embedding_feature('brand', data, 6))
    
    feature_columns.append(set_one_hot_feature('notRepairedDamage', data))
    
    feature_columns.append(set_embedding_feature('postalCode', data, 10))
    

    feature_layer = layers.DenseFeatures(feature_columns)
    
    return(feature_layer)

In [5]:
train = preprocessing_fn(path='data/train.csv', na_encoding='data/gearbox_powerps_na.csv', norm_params='data/numerical_features_normalization.csv')

In [6]:
train_ds = df_to_dataset(dataframe=train, target_name='price', shuffle=True, batch_size=32)

In [8]:
feature_layer = feature_columns(train, train_ds)

In [ ]:
inputs = keras.Input(shape=(784,))